# Сегментация договоров при помощи архитектур: **Линейная Conv1D-сеть**, **PSPnet**, **UNET**

*Разбор данного раздела:* https://youtu.be/OgMO4JFv0qM?t=453

In [ ]:
!pip install pymorphy2 # установка лемматизатора pymorphy2

# Подключение библиотек

*Разбор данного раздела:* https://youtu.be/OgMO4JFv0qM?t=746

In [ ]:
import numpy as np # Библиотека для работы с массивами данных
from tensorflow.keras.models import Model, load_model # Импортируем модель, метод загрузки предобученной сети 
import re # Импортируем для работы со строками
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами и преобразования их в последовательности
# Импортируем слои для сеток
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Lambda
from tensorflow.keras import backend as K # Импортируем для подсчета dice_coef
from tensorflow.keras.optimizers import Adam, Adadelta # Импортируем оптимизаторы 
from tensorflow.keras import utils # Импортируем для работы с категориальными данными 
from google.colab import files # Импортируем для работы с файлами
import matplotlib.pyplot as plt # Импортируем для отрисовки графиков
from gensim.models import word2vec # Импортируем gensim 
import os # Импортируем для работы с файловой системой
import pandas as pd # И мпортируем для работы с DataFrame
import pymorphy2 # Импортируем для работы с леммами
import time # Импортируем для подсчета времени 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Функции для работы с текстом

*Разбор данного раздела:* https://youtu.be/OgMO4JFv0qM?t=825

Преобразуем тесты договоров в списке слов

In [ ]:
# Чтение файла текста из файла, очитска от знаков препинания
def readText(fileName):
  f = open(fileName, 'r') #Открываем наш файл для чтения и считываем из него данные 
  text = f.read() #Записываем прочитанный текст в переменную 
  # Определяем символы для удаления
  delSymbols = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";"]

  for dS in delSymbols: # Каждый символ в списке символов для удаления
    text = text.replace(dS, " ") # Удаляем, заменяя на пробел

  # Ищем шаблоны и символы в строке и меняем на указанную подстроку
  text = re.sub("[.]", " ", text) 
  text = re.sub(":", " ", text)
  text = re.sub("<", " <", text)
  text = re.sub(">", "> ", text)

  # Метод split разделит текст по пробелам (а их может быть несколько после удаления символов)
  # При помощи метода join запишем все разделенные слова снова в строку
  text = ' '.join(text.split()) 

  text = text.lower() # Переводим текст в нижний регистр
  return text # Возвращаем тексты

# Преобразование исходного текста в список из слов в нормальной форме 
def text2Words(text):
  morph = pymorphy2.MorphAnalyzer() # Создаем экземпляр класса MorphAnalyzer
  words = text.split(' ') # Разделяем текст на пробелы
  words = [morph.parse(word)[0].normal_form for word in words] #Переводим каждое слово в нормалную форму  
  return words # Возвращаем слова


# Чтение данных 

*Разбор данного раздела:* https://youtu.be/OgMO4JFv0qM?t=1057

In [ ]:
directory = '/content/drive/My Drive/Базы/Договоры 432/' # Путь к папке с договорами
os.listdir(directory)[250:255] # Выведем названия 5-ти договоров

['dogovor-zaloga-privatizirovannoy-kvartiri.txt',
 'dogovor-zaloga-s-peredachej-predmeta-zaloga-zalogoderzhatelyu.txt',
 'dogovor-zaloga-nedvizhimosti.txt',
 'dogovor-zaloga-imuschestvennih-prav.txt',
 'dogovor-zaloga-imuschestva-kvartiri.txt']

In [ ]:
print('Всего', len(os.listdir(directory)), 'договора.')

Всего 432 договора.


In [ ]:
agreements = [] # Список, в который запишем все наши договоры
for filename in os.listdir(directory): # Проходим по всем файлам в директории договоров
  txt = readText(directory + filename) # Читаем текст договора
  if txt != '': # Если текст не пустой
    agreements.append(readText(directory + filename)) # Преобразуем файл в одну строку и добавляем в agreements

In [ ]:
n = 4
print(os.listdir(directory)[n]) # Выведем название n-ого договора
agreements[n] # Текст n-го договора

2 (4).txt


'договор бартера г г в лице действующего на основании именуемый в дальнейшем сторона 1 с одной стороны и в лице действующего на основании именуемый в дальнейшем сторона 2 с другой стороны именуемые в дальнейшем стороны заключили настоящий договор в дальнейшем договор о нижеследующем 1 предмет договора 1 1 <s1> сторона 1 принимает на себя обязательства поставить стороне 2 в количестве <s3> по цене на общую сумму рублей </s1> </s3> 1 2 <s1> сторона 2 принимает на себя обязательства принять и отгрузить по реквизитам стороны 1 в количестве <s3> по цене на общую сумму рублей </s1> </s3> 1 3 <s1> качество товара должно соответствовать </s1> 2 порядок поставки товара 2 1 <s1> товар передается в месте нахождения принимающей стороны </s1> с этого момента обязательство по поставке и передаче права собственности считается выполненным 2 2 <s3> доставка производится за счет </s3> 2 3 доставка производится 3 сроки поставки и передачи 3 1 <s1> <s4> сторона 1 обязуется поставить стороне 2 в срок до го

In [ ]:
words = [] # Здесь будут храниться все договора в виде списка слов
curTime = time.time() # Засечем текущее время
for i in range(len(agreements)): # Проходимся по всем договорам
  words.append(text2Words(agreements[i])) # Преобразуем очередной договор в список слов и добавляем в words
print('На преобразование ушло:', round(time.time() - curTime, 2), 'с.')

На преобразование ушло: 175.42 с.


In [ ]:
wordsToTest = words[-10:] # Возьмем 10 текстов для финальной проверки обученной нейронной сети 
words = words[:-10] # Для обученающей и проверочной выборок возьмем все тексты, кроме последних 10

In [ ]:
print('Колличество текстов в выборке для финальной проверки в конце ноутбука:', len(wordsToTest)) 
print('Колличество договоров, взятых для обучающей и тестовой выборки:',len(words)) 
print('Колличество слов в первом договоре:', len(words[10])) # Количество слов в первом договоре

Колличество текстов в выборке для финальной проверки в конце ноутбука: 10
Колличество договоров, взятых для обучающей и тестовой выборки: 412
Колличество слов в первом договоре: 503


In [ ]:
print('Пример исходного текста:')
print(agreements[4][:62], '\n')
print('Тот же текст, представленный в виде списка из слов в начальной форме:')
print(words[4][:10])

Пример исходного текста:
договор бартера г г в лице действующего на основании именуемый 

Тот же текст, представленный в виде списка из слов в начальной форме:
['договор', 'бартер', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать']


# Преобразование текста в индексы: формирование xTrain

*Разбор данного раздела:* https://youtu.be/OgMO4JFv0qM?t=1256

In [ ]:
# lower=True - приводим слова к нижнему регистру
# char_level=False - просим токенайзер учитывать слова, а не отдельные символы
tokenizer = Tokenizer(lower=True, filters = '', char_level=False)

tokenizer.fit_on_texts(words) # "Скармливаем" наши тексты, т.е. даём в обработку методу, который соберет словарь частотности
clean_voc = {} 

for item in tokenizer.word_index.items(): #Преобразуем полученный список 
  clean_voc[item[0]] = item[1] # В словарь, меняя местами элементы полученного кортежа 

In [ ]:
print('Словарь всех слов по частоте:') 
print(clean_voc, '\n')
print('Длинна словаря:', len(clean_voc))

tag = '<s1>'
print('Индекс тега', tag, ':' ,clean_voc[tag])

Словарь всех слов по частоте:
{'в': 1, 'договор': 2, 'и': 3, '2': 4, '1': 5, 'по': 6, 'сторона': 7, 'настоящий': 8, 'с': 9, 'на': 10, '3': 11, '<s1>': 12, '</s1>': 13, 'не': 14, '4': 15, 'или': 16, 'за': 17, 'он': 18, 'случай': 19, 'о': 20, '5': 21, '•': 22, '<s3>': 23, '</s3>': 24, 'право': 25, 'работа': 26, 'срок': 27, 'от': 28, '<s4>': 29, '</s4>': 30, 'они': 31, 'при': 32, '6': 33, 'лицо': 34, 'имущество': 35, 'арендатор': 36, 'быть': 37, 'другой': 38, 'день': 39, 'действовать': 40, 'для': 41, 'условие': 42, '7': 43, 'если': 44, 'к': 45, 'п': 46, 'дальнейший': 47, 'который': 48, 'обязательство': 49, 'а': 50, 'страховой': 51, 'сумма': 52, 'порядок': 53, 'законодательство': 54, 'арендодатель': 55, 'работник': 56, 'действие': 57, 'указанный': 58, 'счёт': 59, 'свой': 60, 'размер': 61, 'соответствие': 62, 'именовать': 63, 'из': 64, 'мочь': 65, 'обязать': 66, 'страховщик': 67, 'исполнение': 68, 'один': 69, 'основание': 70, 'страхователь': 71, 'обязанность': 72, 'иной': 73, 'год': 74, 'вп

In [ ]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
tok_agreem = tokenizer.texts_to_sequences(words) # Обучающие тесты в индексы

print("Взглянем на фрагмент обучающего текста:")
print("В виде оригинального текста:              ", words[4][:20])
print("Он же в виде последовательности индексов: ", tok_agreem[4][:20], '\n')

Взглянем на фрагмент обучающего текста:
В виде оригинального текста:               ['договор', 'бартер', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'сторона', '1', 'с', 'один', 'сторона', 'и', 'в', 'лицо']
Он же в виде последовательности индексов:  [2, 4180, 77, 77, 1, 34, 40, 10, 70, 63, 1, 47, 7, 5, 9, 69, 7, 3, 1, 34] 



# Формирование yTrain

*Разбор данного раздела:* https://youtu.be/OgMO4JFv0qM?t=1381


Каждому индексу (слову) из договора необходимо поставить в соответсвие вектор из 6-ти элементов, соответствующих тегам категорий

In [ ]:
# Собираем список индексов и dummy encoded вектора
def get01XSamples(tok_agreem, tags_index):
  tags01 = [] # Список для тегов
  indexes = [] # Здесь будут лежать индексы
 
  for agreement in tok_agreem: # Проходимся по каждому договору-списку
    tag_place = [0, 0, 0, 0, 0, 0] # Создаем вектор [0,0,0,0,0,0]
    for ex in agreement: # Проходимся по каждому слову договора
        if ex in tags_index: # Смотрим, если индекс оказался нашим тегом
          place = np.argwhere(tags_index==ex) # Записываем под каким местом лежит этот тег в своем списке
          if len(place)!=0: # Проверяем, чтобы тег действительно был
            if place[0][0]<6: # Первые шесть тегов в списке - открывающие
              tag_place[place[0][0]] = 1    # Поэтому ставим 1
            else: 
              tag_place[place[0][0] - 6] = 0  # Остальные теги закрывающие, так что меняем на ноль
        else:          
          tags01.append(tag_place.copy()) # Расширяем наш список с каждой итерацией. Получаем в конце длинный список из всех тегов в одном 
          indexes.append(ex) # Докидываем индекс слова в список индексов

  return indexes, tags01

# Получение списка слов из индексов
def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items())) # Берем пары значений всего словаря и размечаем наоборот, т.е. value:key
  words = [reverse_word_map.get(letter) for letter in x] # Вытаскиваем по каждому ключу в список
  return words # Возвращаем полученный текст

In [ ]:
# s1 Условия
# s2 Запреты
# s3 Стоимость, всё про цены и деньги
# s4 Всё про сроки
# s5 Неустойка
# s6 Всё про адреса и геолокации

tags_index = ['<s' + str(i) + '>' for i in range(1, 7)] # Получаем список открывающих тегов
closetags = ['</s' + str(i) + '>' for i in range(1, 7)] # Получаем список закрывающих тегов
tags_index.extend(closetags) # Объединяем все теги

tags_index = np.array([clean_voc[i] for i in tags_index]) # Получаем из словаря частотности индексы всех тегов
print('Индексы тегов:', tags_index)

Индексы тегов: [ 12 350  23  29 118 274  13 358  24  30 119 265]


In [ ]:
xData, yData = get01XSamples(tok_agreem,tags_index) # Распознаем теги и создаем список с ними, с индексами
decoded_text = reverseIndex(clean_voc, xData) # Для создания списков с embedding-ами сначала преобразуем список индексов обратно в слова

In [ ]:
print('Длина xData:', len(xData))
print('Длина yData:', len(yData))

print('Посмотрим на исходный набор слов: ', words[0][50:80])
print('Посмотрим на декодированный текст:', decoded_text[50:80])
print('Посмотрим на часть из xData:     ', xData[50:80])
print('Посмотрим на часть из yData:     ', yData[50:80])

Длина xData: 411870
Длина yData: 411870
Посмотрим на исходный набор слов:  ['1', 'предмет', 'договор', '1', '1', '<s1>', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '</s1>', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по']
Посмотрим на декодированный текст: ['1', 'предмет', 'договор', '1', '1', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор']
Посмотрим на часть из xData:      [5, 108, 2, 5, 5, 6, 8, 2, 220, 115, 914, 315, 124, 18, 240, 3, 539, 242, 315, 1, 411, 5, 4, 240, 299, 10, 159, 6, 8, 2]
Посмотрим на часть из yData:      [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0],

# Разделение выборки на окна

*Разбор данного раздела:* https://youtu.be/OgMO4JFv0qM?t=2580

In [ ]:
# Формируем выборку из индексов
def getSetFromIndexes(wordIndexes, xLen, step): 
  xBatch = [] # Лист для фрагментов текста
  wordsLen = len(wordIndexes) # Определяем длинну текста
  index = 0 # Задаем стартовый индекс
  
  while (index + xLen <= wordsLen): # Пока сумма индекса с длинной фрагмента меньше или равна числу слов в выборке
    xBatch.append(wordIndexes[index:index+xLen]) # Добавляем X в лист фразментов текста
    index += step # Сдвигаемся на step

  return xBatch # Лист для фрагментов текста

In [ ]:
xLen = 256 # Длина окна
step = 30 # Шаг 
embeddingSize = 300 # Количество измерений для векторного пространства

In [ ]:
# Генерируем наборы с заданными параметрами окна
xTrain = getSetFromIndexes(decoded_text, xLen, step) # Последовательность из xLen слов
yTrain = getSetFromIndexes(yData, xLen, step) # Последовательность из xLen-тегов

In [ ]:
print('Длинна xTrain:', len(xTrain))
print('Длинна yTrain:', len(yTrain))
print('Длинна примера из xTrain:',len(xTrain[0]))
print('Длинна примера из yTrain:',len(yTrain[0]), '\n')
print('Пример xTrain', xTrain[0])
print('Пример yTrain', yTrain[0], '\n')

print('Первый пример xTrain:', xTrain[0][step-5:step+5])
print('Второй пример xTrain:', xTrain[1][:10])

Длинна xTrain: 13721
Длинна yTrain: 13721
Длинна примера из xTrain: 256
Длинна примера из yTrain: 256 

Пример xTrain ['договор', 'безвозмездный', 'хранение', 'до', 'востребование', 'г', 'г', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'хранитель', 'с', 'один', 'сторона', 'и', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'поклажедатель', 'с', 'другой', 'сторона', 'именовать', 'в', 'дальнейший', 'сторона', 'заключить', 'настоящий', 'договор', 'в', 'дальнейший', 'договор', 'о', 'нижеследующий', '1', 'предмет', 'договор', '1', '1', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор', 'следующий', 'вещь', 'далее', 'вещь', '1', '3', 'при', 'передача', 'вещь',

# Создание xTrain и  yTrain при помощи метода Word2Wec GENSIM

*Разбор данного раздела:* https://youtu.be/OgMO4JFv0qM?t=2687

In [ ]:
# Создаем выборки
def getSets(model, senI, tagI):
  xVector = [] # Здесь будет лежать embedding представление каждого из индексов
  tmp = [] # Временный список
  for text in senI: # Проходимся по каждому тексту-списку
    tmp=[]
    for word in text: # Проходимся по каждому слову в тексте-списке
      tmp.append(model[word]) 

    xVector.append(tmp)

  return np.array(xVector), np.array(tagI)

In [ ]:
# Передаем в word2vec списки списков слов для обучения
# size = embeddingSize - размер эмбеддинга
# window = 10 - минимальное расстояние между словами в эмбеддинге 
# min_count = 1 - игнорирование всех слов с частотой, меньше, чем 1
# workers = 10 - число потоков обучения эмбеддинга
# iter = 10 - число эпох обучения эмбеддинга

modelGENSIM = word2vec.Word2Vec(xTrain, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10)

In [ ]:
modelGENSIM['слово'] # Пример преобразования слова в вектор при помощи GENSIM

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-7.0524287e-01, -3.2404315e-01,  1.0773668e-01,  1.0223871e+00,
       -1.0688845e+00, -1.7984349e+00,  1.9801735e+00,  9.8870158e-01,
       -3.2311168e-01, -6.5017730e-01,  8.4308892e-01,  1.3906347e+00,
       -1.6067935e+00,  1.3864353e+00, -4.2606056e-01, -5.9284532e-01,
       -7.4180514e-01, -1.6646719e-01, -1.4118154e+00, -7.4165893e-01,
        1.6925994e+00, -1.5249617e+00, -4.2219849e+00, -1.6406186e+00,
       -3.9018261e-01,  1.1479424e+00, -4.5596182e-01,  2.6835063e-01,
        1.9148886e+00,  1.6033756e+00,  3.2771355e-01, -4.2584338e+00,
        1.2128912e+00,  1.5181794e+00, -2.8207304e+00,  2.8826358e+00,
        7.7009958e-01,  2.2738352e-01, -7.0144343e-01,  3.8666332e-01,
        2.7585727e-01,  1.5221163e+00, -2.2077937e+00,  1.4950123e+00,
       -1.7146714e+00,  9.4742000e-01,  5.7305318e-01,  5.7347250e-01,
        9.6624857e-01,  2.9521123e-01,  1.5215628e+00, -1.0811365e+00,
       -8.8498813e-01, -1.6215079e+00, -9.7996220e-02, -4.5185223e-01,
      

In [ ]:
xTrainGENSIM, yTrainGENSIM = getSets(modelGENSIM, xTrain, yTrain)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
print('Размерность полученного xTrain:', xTrainGENSIM.shape)
print('Размерность полученного yTrain:', yTrainGENSIM.shape)

Размерность полученного xTrain: (13721, 256, 300)
Размерность полученного yTrain: (13721, 256, 6)


# Сохранение массивов из оперативной памяти в память runtime 

*Разбор данного раздела:* https://youtu.be/OgMO4JFv0qM?t=3066

**Перед перезагрузкой сессии**

In [ ]:
np.save('xTrainGENSIM.npy', xTrainGENSIM) #Сохраняем массивы в память колаба
np.save('yTrainGENSIM.npy', yTrainGENSIM) #Сохраняем массивы в память колаба

**После перезагрузки сессии**

In [ ]:
xTrainGENSIM = np.load('xTrainGENSIM.npy') #Загружаем массивы из памяти колаба
yTrainGENSIM = np.load('yTrainGENSIM.npy') #Загружаем массивы из памяти колаба

# Функции для создания моделей

## Функция подсчета ошибки

*Разбор данного раздела:* https://youtu.be/OgMO4JFv0qM?t=3200


In [ ]:
# Функция, которая смотрит на пересечение областей. Нужна для accuracy
def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

## **Conv1D**

*Разбор данного раздела:* https://youtu.be/OgMO4JFv0qM?t=3273

In [ ]:
# Функция для создания линейной cnonv1d-сети
def create_Conv1d(xLen, embeddingSize): 
  text_input_layer = Input((xLen,embeddingSize)) 
  text_layer = Conv1D(16, 3, padding='same',activation='relu')(text_input_layer)
  text_layer = Conv1D(16, 3, padding='same',activation='relu')(text_layer)
  text_layer = Conv1D(16, 3,padding='same', activation='relu')(text_layer) 
  text_layer = Conv1D(yTrainGENSIM.shape[-1], 3, padding='same',activation='sigmoid')(text_layer)
  model = Model(text_input_layer, text_layer)
  model.compile(optimizer=Adadelta(),
                    loss='categorical_crossentropy',
                    metrics=[dice_coef])
  return model

## **PSPnet**

*Разбор данного раздела:* https://youtu.be/OgMO4JFv0qM?t=3435

In [ ]:
# Функция для создания PSPnet-сети
def create_PSPNet(conv_size = 64, num_classes = 6, input_shape = (30, 300)):

    # У кераса нет Conv1DTranspose, так что прописываем вручную. Нужен для разворота после maxpooling
    def Conv1DTranspose(input_tensor, filters, kernel_size=1, strides=2, padding='same'):
        """
        input_tensor: входной тензор (batch_size, time_steps, dims)
        filters: int, output dimension, выходной тензор будет иметь размер (batch_size, time_steps, filters)
        kernel_size: размер ядра свертки 
        strides: int, шаг ядра
        padding: 'same' | действительный
        """
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x
        
    img_input = Input(input_shape)

    # Block 1
    x = Conv1D(conv_size, 3, padding='same')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv1D(conv_size, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x_mp_2 = MaxPooling1D(2)(x)
    x_mp_4 = MaxPooling1D(4)(x)
    x_mp_8 = MaxPooling1D(8)(x)
    x_mp_16 = MaxPooling1D(16)(x)
    x_mp_32 = MaxPooling1D(32)(x)

    x_mp_2 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_2)
    x_mp_2 = Dropout(0.5)(x_mp_2)

    x_mp_4 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_4)
    x_mp_4 = Dropout(0.5)(x_mp_4)

    x_mp_8 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_8)
    x_mp_8 = Dropout(0.5)(x_mp_8)

    x_mp_16 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_16)
    x_mp_16 = Dropout(0.5)(x_mp_16)

    x_mp_32 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_32)
    x_mp_32 = Dropout(0.5)(x_mp_32) 

    x_mp_2 = Conv1DTranspose(x_mp_2,conv_size, 2)
    x_mp_2 = Activation('relu')(x_mp_2)

    x_mp_4 = Conv1DTranspose(x_mp_4,conv_size, 4, strides=4)
    x_mp_4 = Activation('relu')(x_mp_4)

    x_mp_8 = Conv1DTranspose(x_mp_8,conv_size, 8, strides=8)
    x_mp_8 = Activation('relu')(x_mp_8)

    x_mp_16 = Conv1DTranspose(x_mp_16,conv_size, 16, strides=16)
    x_mp_16 = Activation('relu')(x_mp_16)

    x_mp_32 = Conv1DTranspose(x_mp_32,conv_size, 32, strides=32)
    x_mp_32 = Activation('relu')(x_mp_32)


    fin = concatenate([img_input, x_mp_2, x_mp_4, x_mp_8, x_mp_16, x_mp_32])    
    fin = Conv1D(conv_size, 3, padding='same')(fin)
    fin = BatchNormalization()(fin)
    fin = Activation('relu')(fin)
    fin = Conv1D(conv_size, 3, padding='same')(fin)
    fin = BatchNormalization()(fin)
    fin = Activation('relu')(fin)

    fin = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(fin)

    model = Model(img_input, fin)
    model.compile(optimizer=Adadelta(lr=0.001),
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
    
    return model

##**UNET**

In [ ]:
#Функция для создания сети UNET 
def create_unet(k = 1, num_classes = 6, input_shape= (30, 300)):

    # У кераса нет Conv1DTranspose, так что прописываем вручную. Нужен для разворота после maxpooling
    def Conv1DTranspose(input_tensor, filters, kernel_size=1, strides=2, padding='same'):
        """
        input_tensor: входной тензор (batch_size, time_steps, dims)
        filters: int, output dimension, выходной тензор будет иметь размер (batch_size, time_steps, filters)
        kernel_size: размер ядра свертки 
        strides: int, шаг ядра
        padding: 'same' | действительный
        """
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x
        
    img_input = Input(input_shape) 

    # Block 1
    x = Conv1D(64 * k , 3, padding='same')(img_input) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)  
    x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    x = MaxPooling1D()(block_1_out)

    # Block 2
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    x = MaxPooling1D()(block_2_out)

    # Block 3
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # x = block_3_out
    x = MaxPooling1D()(block_3_out)

    # Block 4
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(x, 256, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(x, 128, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(x, 64, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model

# Обучение сетей

*Разбор данного раздела:* https://youtu.be/FcrgZreYGqY?t=193

## **Линейная Conv1d-сеть**

In [ ]:
model_conv1d = create_Conv1d(xLen, embeddingSize) # Создаем простую сеть
model_conv1d.summary() # Смотрим полученную архитектуру 

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 300)]        0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 256, 16)           14416     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 256, 16)           784       
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 256, 16)           784       
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 256, 6)            294       
Total params: 16,278
Trainable params: 16,278
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model_conv1d.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=200, validation_split = 0.2) # Скармливаем модели полученные вектора

Epoch 1/20
55/55 [==============================] - 2s 30ms/step - loss: 1.2424 - dice_coef: 0.2236 - val_loss: 1.2948 - val_dice_coef: 0.2189
Epoch 2/20
55/55 [==============================] - 2s 29ms/step - loss: 1.2403 - dice_coef: 0.2242 - val_loss: 1.2930 - val_dice_coef: 0.2194
Epoch 3/20
55/55 [==============================] - 2s 29ms/step - loss: 1.2381 - dice_coef: 0.2247 - val_loss: 1.2910 - val_dice_coef: 0.2199
Epoch 4/20
55/55 [==============================] - 2s 29ms/step - loss: 1.2358 - dice_coef: 0.2254 - val_loss: 1.2890 - val_dice_coef: 0.2205
Epoch 5/20
55/55 [==============================] - 2s 29ms/step - loss: 1.2335 - dice_coef: 0.2260 - val_loss: 1.2869 - val_dice_coef: 0.2210
Epoch 6/20
55/55 [==============================] - 2s 29ms/step - loss: 1.2311 - dice_coef: 0.2267 - val_loss: 1.2848 - val_dice_coef: 0.2216
Epoch 7/20
55/55 [==============================] - 2s 28ms/step - loss: 1.2286 - dice_coef: 0.2274 - val_loss: 1.2827 - val_dice_coef: 0.2222

## **PSPnet**

### Стандартная архитектура

In [ ]:
model_b_PSPnet = create_PSPNet(input_shape=(xLen, embeddingSize))
model_b_PSPnet.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 256, 64)      57664       input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 64)      256         conv1d_8[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 64)      0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
history = model_b_PSPnet.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=64, validation_split = 0.2)

Epoch 1/20
172/172 [==============================] - 5s 27ms/step - loss: 1.3376 - dice_coef: 0.1961 - val_loss: 1.3483 - val_dice_coef: 0.1973
Epoch 2/20
172/172 [==============================] - 4s 23ms/step - loss: 1.3269 - dice_coef: 0.1986 - val_loss: 1.3426 - val_dice_coef: 0.2003
Epoch 3/20
172/172 [==============================] - 4s 23ms/step - loss: 1.3158 - dice_coef: 0.2011 - val_loss: 1.3332 - val_dice_coef: 0.2029
Epoch 4/20
172/172 [==============================] - 4s 23ms/step - loss: 1.3045 - dice_coef: 0.2038 - val_loss: 1.3227 - val_dice_coef: 0.2054
Epoch 5/20
172/172 [==============================] - 4s 23ms/step - loss: 1.2930 - dice_coef: 0.2066 - val_loss: 1.3118 - val_dice_coef: 0.2080
Epoch 6/20
172/172 [==============================] - 4s 23ms/step - loss: 1.2815 - dice_coef: 0.2093 - val_loss: 1.3010 - val_dice_coef: 0.2107
Epoch 7/20
172/172 [==============================] - 4s 23ms/step - loss: 1.2699 - dice_coef: 0.2124 - val_loss: 1.2902 - val_dic

### 256 нейронов на ядре свертки

In [ ]:
model_b_PSPnet = create_PSPNet(conv_size = 256, input_shape=(xLen, embeddingSize))
model_b_PSPnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 256, 256)     230656      input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 256, 256)     1024        conv1d_10[0][0]                  
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 256, 256)     0           batch_normalization_4[0][0]      
____________________________________________________________________________________________

In [ ]:
history = model_b_PSPnet.fit(xTrainGENSIM, yTrainGENSIM, epochs=60, batch_size=64, validation_split = 0.2)

Epoch 1/60
172/172 [==============================] - 14s 83ms/step - loss: 1.2762 - dice_coef: 0.2168 - val_loss: 1.2822 - val_dice_coef: 0.2177
Epoch 2/60
172/172 [==============================] - 13s 75ms/step - loss: 1.2439 - dice_coef: 0.2246 - val_loss: 1.2614 - val_dice_coef: 0.2243
Epoch 3/60
172/172 [==============================] - 13s 75ms/step - loss: 1.2129 - dice_coef: 0.2326 - val_loss: 1.2336 - val_dice_coef: 0.2322
Epoch 4/60
172/172 [==============================] - 13s 75ms/step - loss: 1.1830 - dice_coef: 0.2411 - val_loss: 1.2060 - val_dice_coef: 0.2401
Epoch 5/60
172/172 [==============================] - 13s 74ms/step - loss: 1.1543 - dice_coef: 0.2495 - val_loss: 1.1794 - val_dice_coef: 0.2480
Epoch 6/60
172/172 [==============================] - 13s 75ms/step - loss: 1.1265 - dice_coef: 0.2582 - val_loss: 1.1539 - val_dice_coef: 0.2560
Epoch 7/60
172/172 [==============================] - 13s 75ms/step - loss: 1.1000 - dice_coef: 0.2668 - val_loss: 1.1294 - 

### 512 нейронов на ядре свертки

In [ ]:
model_b_PSPnet = create_PSPNet(conv_size = 512, input_shape=(xLen, embeddingSize))
model_b_PSPnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 256, 512)     461312      input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 256, 512)     2048        conv1d_10[0][0]                  
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 256, 512)     0           batch_normalization_4[0][0]      
____________________________________________________________________________________________

In [ ]:
history = model_b_PSPnet.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=64, validation_split = 0.2)

Epoch 1/20
172/172 [==============================] - 33s 193ms/step - loss: 1.2447 - dice_coef: 0.2233 - val_loss: 1.2552 - val_dice_coef: 0.2216
Epoch 2/20
172/172 [==============================] - 32s 184ms/step - loss: 1.1891 - dice_coef: 0.2386 - val_loss: 1.2040 - val_dice_coef: 0.2392
Epoch 3/20
172/172 [==============================] - 32s 183ms/step - loss: 1.1396 - dice_coef: 0.2536 - val_loss: 1.1595 - val_dice_coef: 0.2538
Epoch 4/20
172/172 [==============================] - 32s 183ms/step - loss: 1.0946 - dice_coef: 0.2682 - val_loss: 1.1197 - val_dice_coef: 0.2671
Epoch 5/20
172/172 [==============================] - 31s 183ms/step - loss: 1.0534 - dice_coef: 0.2826 - val_loss: 1.0834 - val_dice_coef: 0.2800
Epoch 6/20
172/172 [==============================] - 32s 183ms/step - loss: 1.0158 - dice_coef: 0.2965 - val_loss: 1.0503 - val_dice_coef: 0.2925
Epoch 7/20
172/172 [==============================] - 31s 183ms/step - loss: 0.9820 - dice_coef: 0.3102 - val_loss: 1.

## **UNET**

### Стандартная архитектура

In [ ]:
model_b_UNET = create_unet(input_shape=(xLen, embeddingSize))
model_b_UNET.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_18 (Conv1D)              (None, 256, 64)      57664       input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 256, 64)      256         conv1d_18[0][0]                  
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 256, 64)      0           batch_normalization_4[0][0]      
_______________________________________________________________________________________

In [ ]:
history = model_b_UNET.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=64, validation_split = 0.2) # Скармливаем модели изображения, разделенные по 3 классам

Epoch 1/20
172/172 [==============================] - 8s 46ms/step - loss: 0.5737 - dice_coef: 0.5082 - val_loss: 2.5919 - val_dice_coef: 0.3253
Epoch 2/20
172/172 [==============================] - 7s 42ms/step - loss: 0.3640 - dice_coef: 0.6238 - val_loss: 2.2596 - val_dice_coef: 0.3389
Epoch 3/20
172/172 [==============================] - 7s 42ms/step - loss: 0.2920 - dice_coef: 0.6700 - val_loss: 1.1192 - val_dice_coef: 0.5276
Epoch 4/20
172/172 [==============================] - 7s 42ms/step - loss: 0.2684 - dice_coef: 0.6886 - val_loss: 1.2208 - val_dice_coef: 0.5179
Epoch 5/20
172/172 [==============================] - 7s 42ms/step - loss: 0.2527 - dice_coef: 0.7036 - val_loss: 1.3641 - val_dice_coef: 0.5257
Epoch 6/20
172/172 [==============================] - 7s 43ms/step - loss: 0.2408 - dice_coef: 0.7135 - val_loss: 1.3645 - val_dice_coef: 0.5621
Epoch 7/20
172/172 [==============================] - 7s 42ms/step - loss: 0.2463 - dice_coef: 0.7134 - val_loss: 1.3491 - val_dic

# **Проверка работы сети**

*Разбор данного раздела:* https://youtu.be/FcrgZreYGqY?t=259

In [ ]:
# Функция, выводящая точность распознавания каждой категории отдельно
def recognizeSet(tagI, pred, tags, length, value):
  total=0

  for j in range(6): # общее количество тегов
    correct=0
    for i in range(len(tagI)): # проходимся по каждому списку списка тегов
      for k in range(length): # проходимся по каждому тегу
        if tagI[i][k][j]==(pred[i][k][j]>value).astype(int): # если соответствующие индексы совпадают, значит сеть распознала верно
          correct+=1 
    print("Сеть распознала категорию '{}' на {}%".format(tags[j],round(100*correct/(len(tagI)*length), 2)))
    total += 100 * correct / (len(tagI)*length)
  print("Cредняя точность {}%".format(total/6))

In [ ]:
model = model_b_UNET 

In [ ]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
tok_agreemTest = tokenizer.texts_to_sequences(wordsToTest) # Обучающие тесты в индексы

print("Взглянем на фрагмент тестового текста:")
print("В виде оригинального текста:              ", wordsToTest[4][:20])
print("Он же в виде последовательности индексов: ", tok_agreemTest[4][:20], '\n')

Взглянем на фрагмент тестового текста:
В виде оригинального текста:               ['трудовой', 'контракт', 'год', 'год', 'в', 'лицо', 'действующий', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'сторона', '1', 'с', 'один', 'сторона', 'и', 'гражданин', 'паспорт']
Он же в виде последовательности индексов:  [90, 330, 32, 32, 1, 35, 49, 9, 73, 65, 1, 46, 7, 5, 8, 69, 7, 3, 270, 249] 



In [ ]:
xDataTest, yDataTest = get01XSamples(tok_agreemTest,tags_index) # Распознаем теги и создаем список с ними, с индексами
decoded_text = reverseIndex(clean_voc, xDataTest) # Для создания списков с embedding-ами сначала преобразуем список индексов обратно в слова

In [ ]:
print('Длина xDataTest:', len(xDataTest))
print('Длина yDataTest:', len(yDataTest))

print('Посмотрим на исходный набор слов: ', wordsToTest[0][50:80])
print('Посмотрим на декодированный текст:', decoded_text[50:80])
print('Посмотрим на часть из xDataTest:     ', xDataTest[50:80])
print('Посмотрим на часть из yDataTest:     ', yDataTest[50:80])

Длина xDataTest: 7818
Длина yDataTest: 7818
Посмотрим на исходный набор слов:  ['нижеследующий', 'настоящий', 'контракт', 'регулировать', 'трудовой', 'и', 'иной', 'отношение', 'между', 'работник', 'и', 'работодатель', '<s1>', 'договариваться', 'сторона', 'признавать', 'что', 'они', 'право', 'и', 'обязанность', 'регулироваться', 'настоящее', 'контракт', 'а', 'также', 'действующий', 'коллективный', 'соглашение', 'договор']
Посмотрим на декодированный текст: ['нижеследующий', 'настоящий', 'контракт', 'регулировать', 'трудовой', 'и', 'иной', 'отношение', 'между', 'работник', 'и', 'работодатель', 'договариваться', 'сторона', 'признавать', 'что', 'они', 'право', 'и', 'обязанность', 'регулироваться', 'настоящее', 'контракт', 'а', 'также', 'действующий', 'коллективный', 'соглашение', 'договор', 'правило']
Посмотрим на часть из xDataTest:      [201, 10, 330, 1093, 90, 3, 77, 257, 179, 58, 3, 136, 1537, 7, 1661, 121, 59, 25, 3, 75, 1359, 124, 330, 50, 83, 49, 517, 96, 2, 178]
Посмотрим на часть 

In [ ]:
xLen = 256 # Длина окна
step = 30 # Шаг 
embeddingSize = 300 # Количество измерений для векторного пространства

# Генерируем наборы с заданными параметрами окна
xTest = getSetFromIndexes(decoded_text, xLen, step) # Последовательность из xLen слов
yTest = getSetFromIndexes(yDataTest, xLen, step) # Последовательность из xLen-тегов

In [ ]:
print('Длинна xTest:', len(xTest))
print('Длинна yTest:', len(yTest))
print('Длинна примера из xTest:',len(xTest[0]))
print('Длинна примера из yTrain:',len(yTest[0]), '\n')
print('Пример xTest', xTest[0])
print('Пример yTest', yTest[0], '\n')

print('Первый пример xTest:', xTest[0][step-5:step+5])
print('Второй пример xTest:', xTest[1][:10])

Длинна xTest: 253
Длинна yTest: 253
Длинна примера из xTest: 256
Длинна примера из yTrain: 256 

Пример xTest ['трудовой', 'договор', 'с', 'управлять', 'руководитель', 'структурный', 'подразделение', 'акционерный', 'общество', 'год', 'год', 'в', 'лицо', 'действующий', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'работодатель', 'с', 'один', 'сторона', 'и', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'работник', 'с', 'другой', 'сторона', 'именовать', 'в', 'дальнейший', 'сторона', 'заключить', 'настоящий', 'договор', 'в', 'дальнейший', 'договор', 'о', 'нижеследующий', 'настоящий', 'контракт', 'регулировать', 'трудовой', 'и', 'иной', 'отношение', 'между', 'работник', 'и', 'работодатель', 'договариваться', 'сторона', 'признавать', 'что', 'они', 'право', 'и', 'обязанность', 'регулироваться', 'настоящее', 'контракт', 'а', 'также', 'действующий', 'коллективный', 'соглашение', 'договор', 'правило', 'трудовой', 'распорядо

In [ ]:
# Передаем в word2vec списки списков слов для обучения
# size = embeddingSize - размер эмбеддинга
# window = 10 - расстояние между текущим и прогнозируемым словом в предложении
# min_count = 1 - игнорирование всех слов с частотой, меньше, чем 1
# workers = 10 - число потоков обучения эмбеддинга
# iter = 10 - число эпох обучения эмбеддинга

modelGENSIM = word2vec.Word2Vec(xTest, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10)

In [ ]:
xTestGENSIM, yTestGENSIM = getSets(modelGENSIM, xTest, yTest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
print('Размерность полученного xTrain:', xTestGENSIM.shape)
print('Размерность полученного yTrain:', yTestGENSIM.shape)

Размерность полученного xTrain: (253, 256, 300)
Размерность полученного yTrain: (253, 256, 6)


In [ ]:
tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
# s1 Условия
# s2 Запреты
# s3 Стоимость, всё про цены и деньги
# s4 Всё про сроки
# s5 Неустойка
# s6 Всё про адреса и геолокации

In [ ]:
pred = model.predict(xTestGENSIM) # сделаем предсказание

In [ ]:
recognizeSet(yTestGENSIM, pred, tags, xLen, 0.999)

Сеть распознала категорию 'S1' на 24.48%
Сеть распознала категорию 'S2' на 99.14%
Сеть распознала категорию 'S3' на 83.4%
Сеть распознала категорию 'S4' на 87.98%
Сеть распознала категорию 'S5' на 92.1%
Сеть распознала категорию 'S6' на 99.81%
Cредняя точность 81.151957756917%
